In this notebook, the downloaded data from the human connectome is exctracted and processed before building the final matrices used for the machine learning analysis.

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
import subprocess
from os.path import join as opj

In [ ]:
data_dir = "path/to/some/data/dir"

In [ ]:
cole_atlas = opj("../data", "CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
glasser_atlas = opj("../data", "Q1-Q6_RelatedParcellation210.CorticalAreas_dil_Colors.32k_fs_LR.dlabel.nii")

Load the final subset of subjects to use

In [ ]:
subjects = pd.read_csv(opj(data_dir, "subjects_finally_used.txt"), header=None).values.flatten()

In [ ]:
def get_resting_files(sub_id):
    
    _clean_ts = opj(data_dir, "last", "sub-" + str(sub_id), 
                 "rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.dtseries.nii")
    _parcel_ts = opj(data_dir, "final", "functional", "sub-" + str(sub_id), 
                 "rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.ptseries.nii")
    _parcel_conn_cifti = opj(data_dir, "final", "functional", "sub-" + str(sub_id), 
                 "rfMRI_REST1_LR_Atlas_MSMAll_hp2000_clean.pconn.nii")
    _parcel_conn_txt = opj(data_dir, "final", "functional", "sub-" + str(sub_id), 
                 "connectome.txt")
    
    return ( _clean_ts, _parcel_ts, _parcel_conn_cifti, _parcel_conn_txt )

In [ ]:
def get_structural(sub_id):
    
    _cor_surface = opj(data_dir, "last", "sub-" + str(sub_id), 
                       str(sub_id) + ".midthickness_MSMAll_va.32k_fs_LR.dscalar.nii")
    _cor_thickness = opj(data_dir, "last", "sub-" + str(sub_id), 
                         str(sub_id) + ".thickness_MSMAll.32k_fs_LR.dscalar.nii")
    _sub_volumes = opj(data_dir, "last", "sub-" + str(sub_id), 
                       "aseg.stats")
    
    return ( _cor_surface, _cor_thickness, _sub_volumes)

Process and rearrange functional data

In [ ]:
for sub_id in subjects:
    
    if os.path.exists(opj(data_dir, "final", "functional", "sub-" + str(sub_id))) is False:
        os.mkdir(opj(data_dir, "final", "functional", "sub-" + str(sub_id)))
                      
    clean_ts, parcel_ts, parcel_conn_cifti, parcel_conn_txt = get_resting_files(sub_id)
    
    cmd = ["wb_command", "-cifti-parcellate", clean_ts, atlas_img, "COLUMN", parcel_ts, "-method", "MEAN"]
    subprocess.run(cmd)
    pass

    cmd = ["wb_command", "-cifti-correlation", parcel_ts, parcel_conn_cifti, "-fisher-z"]
    subprocess.run(cmd)
    pass

    cmd = ["wb_command", "-cifti-convert", "-to-text", parcel_conn_cifti, parcel_conn_txt]
    subprocess.run(cmd)
    pass


Process and rearrange structural data

In [ ]:
for sub_id in subjects:
    
    if os.path.exists(opj(data_dir, "final", "structural", "sub-" + str(sub_id))) is False:
        os.mkdir(opj(data_dir, "final", "structural", "sub-" + str(sub_id)))
                      
    cor_surface, cor_thickness, sub_volumes = get_structural(sub_id)
    
    # SURFACE FEATURES
    surface_cifti = opj(data_dir, "final", "structural", "sub-" + str(sub_id), "surface.pscalar.nii")
    output_surface = opj(data_dir, "final", "structural", "sub-" + str(sub_id), "surface.txt")
                         
    cmd = ["wb_command", "-cifti-parcellate", cor_surface, glasser_atlas, "COLUMN", surface_cifti, "-method", "SUM"]
    subprocess.run(cmd)                         
    cmd = ["wb_command", "-cifti-convert","-to-text", surface_cifti, output_surface]
    subprocess.run(cmd)
    pass

    # THICKNESS FEATURES
    thickness_cifti = opj(data_dir, "final", "structural", "sub-" + str(sub_id), "thickness.pscalar.nii")
    output_thickness = opj(data_dir, "final", "structural", "sub-" + str(sub_id), "thickness.txt")
    
    cmd = ["wb_command", "-cifti-parcellate", cor_thickness, glasser_atlas, "COLUMN", thickness_cifti, 
           "-cifti-weights", cor_surface, "-method", "MEAN"]
    subprocess.run(cmd)                         
    cmd = ["wb_command", "-cifti-convert","-to-text", thickness_cifti, output_thickness]
    subprocess.run(cmd)
    pass

    # SUBCORTICAL VOLUMES
    output_sub_vols = opj(data_dir, "final", "structural", "sub-" + str(sub_id), "sub_volumes.txt")

    cmd = ["python2.7", "/usr/local/freesurfer/bin/asegstats2table", "-i", sub_volumes,
           "--meas", "volume", "--transpose", "--tablefile", output_sub_vols]
    subprocess.run(cmd)
    pass
